**If you lost points on the last checkpoint you can get them back by responding to TA/IA feedback**  

Update/change the relevant sections where you lost those points, make sure you respond on GitHub Issues to your TA/IA to call their attention to the changes you made here.

Please update your Timeline... no battle plan survives contact with the enemy, so make sure we understand how your plans have changed.

# COGS 108 - EDA Checkpoint

# Names

- Jiaqi Wu
- Ella Wen
- Zihao Yang
- Yunqi Zhang
- Zhining Zhang

# Research Question

#### How do factors such as course difficulty, instructor quality, and course level (lower vs. upper division) correlate with student satisfaction scores in UCSD STEM courses, based on CAPE data from the 2020-2022 academic year?

The factors that may influence the likelihood of students recommending a class include:

- Instructor Quality: Indicated by the percentage of students recommending the instructor.

- Course Difficulty: Implied by the average number of hours spent studying per week.

- Grade Expectations vs. Actual Grades: Measured by the difference between expected and received GPA.

- Number of Evaluations: A higher number provides a more reliable recommendation rate.

- Enrollment Size: Larger class sizes may impact instruction quality and student engagement.

- Term and Course Context: Course content, teaching methods, and term-specific factors may influence recommendations.

## Background and Prior Work


When students at UCSD register for courses on WebReg each quarter, we often rely on Course and Professor Evaluations (CAPEs) to make our decisions. This online evaluation provides us insight on various aspects of a course, such as instructor, workload, and expected vs. received grades. One of the key categories within CAPE is the recommendation rate of the class, which is the percentage of students who would recommend a course to others. However, it remains unclear which factors have the most significant impact on a student’s likelihood of recommending a course.

For this project, we aim to analyze the correlation between CAPE recommendation rates and other CAPEs factors, such as instructor ratings, expected vs. received grades, study hours per week, and overall course enrollment. This can help us better select the courses.

In this past project shown in Reference #1, a group of UCSD students in COGS108 conducted research on past CAPE data, focusing on identifying the key factors that influence the average GPA of a class at UCSD. Their study explored correlations between expected and actual GPAs, study time, class/professor evaluations, and GPA trends across different fields and course levels. One of their notable findings was that upper-division classes tend to have higher average GPAs than lower-division classes, challenging common assumptions about course difficulty.

While their project primarily aimed to understand GPA predictors and their implications for university policies, our research takes a different approach by focusing on the factors that influence students' likelihood of recommending a class in CAPEs. Instead of analyzing the average GPA received by students, we examine the recommendation rate of a class to understand what aspects contribute to a positive course evaluation. However, like their study, our research also relies on CAPE survey data, making their findings a relevant reference for understanding broader academic trends at UCSD.

Prior studies in Reference #2 have explored various aspects of student course selection. Dahl et al. (2022) examined how student attitudes towards the class, class recommendation rate, and perceived behavioral control impact their decision to enroll in a course. This provides a similar approach to our question, what factors influenced the class recommendation rate. It aligns with our hypothesis that instructor quality, expected workload, and grading play a major role in student's satisfication. 



References (include links):
- 1) https://github.com/COGS108/FinalProjects-Sp23/blob/main/FinalProjectGroup_Sp23_DigitSapiens.ipynb
- 2) https://nactajournal.org/index.php/nactaj/article/view/138
- 3) https://github.com/UCSD-Historical-Enrollment-Data/UCSDHistEnrollData.git/ 

# Hypothesis



- Include your team's hypothesis
- Ensure that this hypothesis is clear to readers
- Explain why you think this will be the outcome (what was your thinking?)

What is your main hypothesis/predictions about what the answer to your question is? Briefly explain your thinking. (2-3 sentences)

# Data

## Data overview

For each dataset include the following information
- Dataset #1
  - Dataset Name:
  - Link to the dataset:
  - Number of observations:
  - Number of variables:
- Dataset #2 (if you have more than one!)
  - Dataset Name:
  - Link to the dataset:
  - Number of observations:
  - Number of variables:
- etc

Now write 2 - 5 sentences describing each dataset here. Include a short description of the important variables in the dataset; what the metrics and datatypes are, what concepts they may be proxies for. Include information about how you would need to wrangle/clean/preprocess the dataset

If you plan to use multiple datasets, add a few sentences about how you plan to combine these datasets.

## Dataset #1 (use name instead of number here)

In [ ]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION 

## Dataset #2 (if you have more than one, use name instead of number here)

In [ ]:
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION 

# Results

## Exploratory Data Analysis

Carry out whatever EDA you need to for your project.  Because every project will be different we can't really give you much of a template at this point. But please make sure you describe the what and why in text here as well as providing interpretation of results and context.

### Section 1 of EDA - please give it a better title than this

Some more words and stuff.  Remember notebooks work best if you interleave the code that generates a result with properly annotate figures and text that puts these results into context.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

### Section 2 of EDA if you need it  - please give it a better title than this

Some more words and stuff.  Remember notebooks work best if you interleave the code that generates a result with properly annotate figures and text that puts these results into context.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

- Thoughtful discussion of ethical concerns included
- Ethical concerns consider the whole data science process (question asked, data collected, data being used, the bias in data, analysis, post-analysis, etc.)
- How your group handled bias/ethical concerns clearly described

Acknowledge and address any ethics & privacy related issues of your question(s), proposed dataset(s), and/or analyses. Use the information provided in lecture to guide your group discussion and thinking. If you need further guidance, check out [Deon's Ethics Checklist](http://deon.drivendata.org/#data-science-ethics-checklist). In particular:

- Are there any biases/privacy/terms of use issues with the data you propsed?
- Are there potential biases in your dataset(s), in terms of who it composes, and how it was collected, that may be problematic in terms of it allowing for equitable analysis? (For example, does your data exclude particular populations, or is it likely to reflect particular human biases in a way that could be a problem?)
- How will you set out to detect these specific biases before, during, and after/when communicating your analysis?
- Are there any other issues related to your topic area, data, and/or analyses that are potentially problematic in terms of data privacy and equitable impact?
- How will you handle issues you identified?

# Team Expectations 


Read over the [COGS108 Team Policies](https://github.com/COGS108/Projects/blob/master/COGS108_TeamPolicies.md) individually. Then, include your group’s expectations of one another for successful completion of your COGS108 project below. Discuss and agree on what all of your expectations are. Discuss how your team will communicate throughout the quarter and consider how you will communicate respectfully should conflicts arise. By including each member’s name above and by adding their name to the submission, you are indicating that you have read the COGS108 Team Policies, accept your team’s expectations below, and have every intention to fulfill them. These expectations are for your team’s use and benefit — they won’t be graded for their details.

* *Team Expectation 1*
* *Team Expectation 2*
* *Team Expecation 3*
* ...

# Project Timeline Proposal

Specify your team's specific project timeline. An example timeline has been provided. Changes the dates, times, names, and details to fit your group's plan.

If you think you will need any special resources or training outside what we have covered in COGS 108 to solve your problem, then your proposal should state these clearly. For example, if you have selected a problem that involves implementing multiple neural networks, please state this so we can make sure you know what you’re doing and so we can point you to resources you will need to implement your project. Note that you are not required to use outside methods.



| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  1 PM | Read & Think about COGS 108 expectations; brainstorm topics/questions  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/26  |  10 AM |  Do background research on topic | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 2/1  | 10 AM  | Edit, finalize, and submit proposal; Search for datasets  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 2/14  | 6 PM  | Import & Wrangle Data (Ant Man); EDA (Hulk) | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/23  | 12 PM  | Finalize wrangling/EDA; Begin Analysis (Iron Man; Thor) | Discuss/edit Analysis; Complete project check-in |
| 3/13  | 12 PM  | Complete analysis; Draft results/conclusion/discussion (Wasp)| Discuss/edit full project |
| 3/20  | Before 11:59 PM  | NA | Turn in Final Project & Group Project Surveys |